In [1]:
import torch
import seaborn as sea
import torch.nn.functional as F
import torch.nn as nn
import numpy as np
import pandas as pd
import time
import re
import random
from scipy.stats import poisson
import datetime
import matplotlib.pyplot as plt
%run functions.ipynb
from unidecode import unidecode
plt.rcParams['figure.figsize'] = [15, 8]
pd.set_option("display.precision", 3)

In [3]:
opp_stats = pd.read_csv('opponent_stats',index_col=0)
schedule = pd.read_csv('schedule',index_col=0)
data = pd.read_csv('data.csv')

lines10 =pd.read_csv('Lines/betting_lines_10',index_col=0)

In [65]:
for i in range(1, 20):
    file_name = f"Lines/lines_{i}"
    try:
        # Use globals() to dynamically set the variable name
        globals()[f"lines{i}"] = pd.read_csv(file_name, index_col=0)
    except FileNotFoundError:
        print(f"{file_name} not found, stopping iteration.")
        break

Lines/betting_lines_15 not found, stopping iteration.


In [69]:
lines11['Date'] = '2023-03-04'

In [81]:
#lines5 is junk
for i in range(1,20):
    try:
        date = globals()[f'lines{i}'].Date[0].split('-')
        month = str(int(date[1]))
        day = str(int(date[2]))
        print(month,day)
        globals()[f'lines{i}'].to_csv(f'Lines_archive/lines{month}_{day}')
        
    except:
        print('oopsies')

2 15
2 16
2 23
2 24
oopsies
2 26
2 27
2 28
3 1
3 2
3 4
3 5
3 6
3 6
oopsies
oopsies
oopsies
oopsies
oopsies


In [4]:
data

,Player,G,Date,Age,Tm,H/A,Opp,W/L,GS,MP,...,AST,STL,BLK,TOV,PF,PTS,GmSc,+/-,Pos,KM
0,Jayson Tatum,1,2022-10-18,24-229,BOS,1,PHI,9,1,38.633,...,4,1,1,3,1,35,30.1,10.0,PF,7
1,Jayson Tatum,2,2022-10-21,24-232,BOS,0,MIA,7,1,36.783,...,4,1,2,2,1,29,22.7,-1.0,PF,7
2,Jayson Tatum,3,2022-10-22,24-233,BOS,0,ORL,6,1,37.650,...,1,1,2,1,2,40,34.2,6.0,PF,7
3,Jayson Tatum,4,2022-10-24,24-235,BOS,0,CHI,-18,1,35.433,...,5,0,0,1,2,26,20.7,-8.0,PF,7
4,Jayson Tatum,5,2022-10-28,24-239,BOS,1,CLE,-9,1,42.183,...,4,1,1,4,4,32,24.1,-10.0,PF,7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25141,Marko Simonovic,3,2023-02-16,23-124,CHI,1,MIL,-12,0,1.800,...,0,0,0,0,1,0,-0.4,8.0,C,15
25142,Marko Simonovic,4,2023-03-31,23-167,CHI,0,CHA,30,0,3.967,...,0,0,0,0,1,0,-0.4,-1.0,C,15
25143,Marko Simonovic,5,2023-04-04,23-171,CHI,1,ATL,-18,0,2.100,...,0,0,0,0,0,2,1.0,0.0,C,15
25144,Marko Simonovic,6,2023-04-05,23-172,CHI,0,MIL,-13,0,0.650,...,0,0,0,0,0,0,0.0,3.0,C,15


In [107]:
data.Tm.unique()

array(['BOS', 'PHI', 'OKC', 'DAL', 'NYK', 'POR', 'MIN', 'MIL', 'ATL',
       'CLE', 'CHI', 'SAC', 'UTA', 'DEN', 'TOR', 'GSW', 'BKN', 'HOU',
       'MEM', 'MIA', 'WAS', 'PHX', 'LAL', 'NOP', 'ORL', 'LAC', 'CHA',
       'SAS', 'DET', 'IND'], dtype=object)

In [109]:
data['PTS_miss'] = 0
for team in data.Tm.unique():
    #
    tm_games = data.loc[data.Tm== team]
    avg = tm_games.groupby('Player').mean().sort_values(by='PTS',ascending = False)
    avg = avg.reset_index()
    #to know who didnt play this game

    dates = tm_games['Date'].unique()
    d2miss = {}
    for d in dates:
        gm = tm_games.loc[tm_games.Date==d]
        merged = gm.merge(avg, on='Player', how='outer', indicator=True)
        missing = merged.loc[merged._merge=='right_only'].Player.values
        missing
        ptmiss = avg.loc[avg.Player.isin(missing)].sum().PTS
        mpmiss = avg.loc[avg.Player.isin(missing)].sum().MP
        d2miss[d] = ptmiss*240/mpmiss
    data.loc[data.Tm==team, 'PTS_miss'] = tm_games['Date'].map(d2miss)
    

In [112]:
data.sort_values(by='PTS_miss')
data.to_csv('data_test.csv',index=False)

In [99]:
data.loc[data.Player=='Jordan Poole'].sort_values(by='PTS')

,Player,G,Date,Age,Tm,H/A,Opp,W/L,GS,MP,...,STL,BLK,TOV,PF,PTS,GmSc,+/-,Pos,KM,PTS_miss
1255,Jordan Poole,11,2022-11-07,23-141,GSW,1,SAC,3,0,21.817,...,0,0,3,2,2,-1.8,-11.0,PG,2,91.949
1259,Jordan Poole,15,2022-11-16,23-150,GSW,0,PHX,-11,0,26.750,...,0,1,1,2,2,4.0,-2.0,PG,2,95.641
1297,Jordan Poole,53,2023-02-04,23-230,GSW,1,DAL,6,0,26.750,...,0,0,4,3,3,-4.4,-4.0,PG,2,87.963
1317,Jordan Poole,73,2023-03-20,23-274,GSW,0,HOU,13,0,26.367,...,1,1,2,0,5,5.7,13.0,PG,2,102.100
1246,Jordan Poole,2,2022-10-21,23-124,GSW,1,DEN,-5,0,27.333,...,3,0,4,0,7,8.1,-20.0,PG,2,82.044
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1258,Jordan Poole,14,2022-11-14,23-148,GSW,1,SAS,37,1,29.133,...,3,1,0,1,36,32.1,33.0,PG,2,110.235
1270,Jordan Poole,26,2022-12-07,23-171,GSW,0,UTA,-1,1,36.817,...,0,0,6,1,36,24.3,4.0,PG,2,117.627
1299,Jordan Poole,55,2023-02-08,23-234,GSW,0,POR,-3,1,34.883,...,0,0,2,2,38,28.5,0.0,PG,2,121.340
1281,Jordan Poole,37,2022-12-30,23-194,GSW,1,POR,6,1,36.167,...,0,0,7,4,41,27.6,18.0,PG,2,130.852


In [66]:
dates = bos['Date'].unique()
d2miss = {}
for d in dates:
    gm = bos.loc[bos.Date==d]
    merged = gm.merge(avg, on='Player', how='outer', indicator=True)
    missing = merged.loc[merged._merge=='right_only'].Player.values
    missing
    d2miss[d] = avg.loc[avg.Player.isin(missing)].sum().PTS


In [69]:
bos['PTS_miss'] = bos['Date'].map(d2miss)
bos

C:\Users\User\AppData\Local\Temp\ipykernel_724\3286755003.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  bos['PTS_miss'] = bos['Date'].map(d2miss)


,Player,G,Date,Age,Tm,H/A,Opp,W/L,GS,MP,...,STL,BLK,TOV,PF,PTS,GmSc,+/-,Pos,KM,PTS_miss
0,Jayson Tatum,1,2022-10-18,24-229,BOS,1,PHI,9,1,38.633,...,1,1,3,1,35,30.1,10.0,PF,7,23.780
1,Jayson Tatum,2,2022-10-21,24-232,BOS,0,MIA,7,1,36.783,...,1,2,2,1,29,22.7,-1.0,PF,7,27.986
2,Jayson Tatum,3,2022-10-22,24-233,BOS,0,ORL,6,1,37.650,...,1,2,1,2,40,34.2,6.0,PF,7,25.143
3,Jayson Tatum,4,2022-10-24,24-235,BOS,0,CHI,-18,1,35.433,...,0,0,1,2,26,20.7,-8.0,PF,7,14.515
4,Jayson Tatum,5,2022-10-28,24-239,BOS,1,CLE,-9,1,42.183,...,1,1,4,4,32,24.1,-10.0,PF,7,28.383
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25025,JD Davison,8,2023-03-17,20-165,BOS,0,POR,14,0,0.717,...,0,0,0,0,0,0.0,0.0,PG,15,35.036
25026,JD Davison,9,2023-03-24,20-172,BOS,1,IND,25,0,2.033,...,0,0,0,0,0,0.7,3.0,PG,15,26.826
25027,JD Davison,10,2023-03-26,20-174,BOS,1,SAS,44,0,6.567,...,0,0,1,2,5,4.3,11.0,PG,15,37.887
25104,Mfiondu Kabengele,1,2022-11-28,25-106,BOS,1,CHA,35,0,8.300,...,0,0,0,1,2,3.0,-2.0,PF,15,49.900


In [70]:
bos.sort_values(by='PTS_miss')

,Player,G,Date,Age,Tm,H/A,Opp,W/L,GS,MP,...,STL,BLK,TOV,PF,PTS,GmSc,+/-,Pos,KM,PTS_miss
22040,Blake Griffin,17,2023-01-05,33-295,BOS,0,DAL,29,0,2.717,...,0,0,0,0,0,0.3,1.0,C,14,7.631
19451,Robert Williams,8,2023-01-05,25-080,BOS,0,DAL,29,0,21.383,...,0,1,0,1,3,5.1,8.0,C,13,7.631
11602,Grant Williams,38,2023-01-05,24-036,BOS,0,DAL,29,0,27.883,...,1,0,0,3,12,11.2,25.0,PF,12,7.631
12071,Al Horford,30,2023-01-05,36-216,BOS,0,DAL,29,1,23.600,...,1,0,1,3,8,8.7,8.0,C,11,7.631
6604,Malcolm Brogdon,33,2023-01-05,30-025,BOS,0,DAL,29,0,24.750,...,0,0,1,2,15,12.5,22.0,PG,8,7.631
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19818,Luke Kornet,50,2023-02-14,27-214,BOS,0,MIL,-6,0,1.600,...,0,0,0,1,0,-0.4,-3.0,C,9,82.158
6696,Derrick White,58,2023-02-14,28-227,BOS,0,MIL,-6,1,43.067,...,3,0,3,4,27,21.6,-5.0,SG,0,82.158
11621,Grant Williams,57,2023-02-14,24-076,BOS,0,MIL,-6,1,47.517,...,1,0,5,5,12,6.8,9.0,PF,12,82.158
22048,Blake Griffin,25,2023-02-14,33-335,BOS,0,MIL,-6,1,24.200,...,1,1,1,3,15,16.6,15.0,C,14,82.158


In [18]:
from sklearn.cluster import KMeans

In [19]:
p_avg = data.groupby('Player').mean()
p_min_avg = data.groupby('Player').mean()

In [20]:
p_avg = p_avg.reset_index()

In [21]:
p_avg

,Player,G,H/A,W/L,GS,MP,FG,FGA,FG%,3P,...,TRB,AST,STL,BLK,TOV,PF,PTS,GmSc,+/-,KM
0,A.J. Green,16.5,0.531,5.688,0.000,9.435,1.500,3.406,0.348,1.219,...,1.188,0.594,0.188,0.000,0.281,0.906,4.344,2.938,-0.875,14.0
1,A.J. Lawson,7.0,0.308,-2.154,0.000,4.818,1.000,2.000,0.467,0.538,...,0.769,0.077,0.077,0.000,0.077,0.615,2.615,1.654,-1.077,0.0
2,AJ Griffin,31.0,0.492,-0.738,0.180,19.370,3.426,7.311,0.462,1.377,...,2.016,0.984,0.639,0.115,0.557,1.213,8.754,6.128,-0.115,8.0
3,Aaron Gordon,29.5,0.569,5.034,1.000,30.099,6.431,11.052,0.587,0.931,...,6.690,2.879,0.776,0.741,1.500,1.879,16.793,14.936,8.276,1.0
4,Aaron Holiday,27.5,0.519,0.074,0.093,13.561,1.463,3.500,0.437,0.574,...,1.222,1.222,0.500,0.185,0.556,1.259,3.944,2.998,-0.222,14.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
498,Zach Collins,28.5,0.536,-8.143,0.339,22.064,4.196,8.054,0.541,0.786,...,6.214,2.786,0.536,0.696,1.911,3.107,10.821,9.068,-4.071,3.0
499,Zach LaVine,32.5,0.531,0.828,1.000,36.037,8.656,18.109,0.473,2.781,...,4.766,4.000,0.938,0.219,2.578,2.172,24.922,17.456,0.156,12.0
500,Zeke Nnaji,22.5,0.568,3.909,0.114,13.264,1.977,3.455,0.529,0.295,...,2.318,0.250,0.341,0.432,0.455,1.841,5.068,4.109,-2.023,14.0
501,Ziaire Williams,18.5,0.472,5.222,0.083,14.669,2.083,4.917,0.419,0.611,...,2.083,0.722,0.389,0.139,1.000,1.583,5.167,2.664,-1.889,14.0


In [37]:
p_avg = p_avg[['Player','FG', 'FG%', '3P', '3P%', 'FT', 'FT%', 'ORB', 'DRB', 'TRB', 'AST', 'STL',
       'BLK', 'TOV','PTS']]
p_avg = p_avg.sort_values(by='PTS',ascending=False)
p_avg = p_avg.dropna()

In [38]:
p_avg['PTS'].max()

33.45454545454545

In [39]:
p_avg['PTS'].min()

0.5

In [40]:
p_avg.iloc[:,1:] = p_avg.iloc[:,1:].astype(float)
p_norm=(p_avg.iloc[:,1:]-p_avg.iloc[:,1:].min())/(p_avg.iloc[:,1:].max()-p_avg.iloc[:,1:].min())

In [41]:
p_norm

,FG,FG%,3P,3P%,FT,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PTS
238,0.980,0.636,0.216,0.314,1.000,0.852,0.353,0.873,0.807,0.377,0.535,0.553,0.859,1.000
320,0.975,0.566,0.549,0.334,0.798,0.715,0.165,0.811,0.684,0.741,0.702,0.161,0.902,0.985
88,0.852,0.523,0.847,0.360,0.848,0.918,0.146,0.414,0.374,0.669,0.421,0.095,0.796,0.964
160,0.986,0.626,0.159,0.294,0.814,0.646,0.446,1.000,0.942,0.506,0.359,0.256,0.966,0.938
433,0.915,0.597,0.175,0.375,0.946,0.904,0.180,0.403,0.380,0.510,0.825,0.332,0.709,0.933
225,0.867,0.521,0.649,0.335,0.707,0.861,0.217,0.819,0.711,0.436,0.482,0.246,0.723,0.899
438,0.890,0.582,1.000,0.440,0.478,0.899,0.129,0.574,0.489,0.578,0.471,0.119,0.816,0.898
315,1.000,0.582,0.422,0.279,0.468,0.717,0.238,0.753,0.669,0.640,0.451,0.181,0.770,0.881
293,0.922,0.680,0.368,0.361,0.659,0.937,0.070,0.668,0.537,0.479,0.379,0.475,0.832,0.879
122,0.892,0.554,0.442,0.376,0.505,0.858,0.178,0.379,0.360,0.515,0.437,0.122,0.624,0.822


In [42]:
clust = KMeans(n_clusters=15, random_state=0).fit(p_avg.iloc[:,1:])

In [43]:
clust.labels_

array([ 7,  7,  7,  7,  7,  7,  7,  7,  7,  3,  3,  3,  3,  3,  3, 10,  3,
        3, 10, 10,  3,  3,  3, 10,  3,  3,  3,  3,  3,  3,  3, 10,  3,  2,
        3,  2,  2,  2,  2,  2, 10,  2,  2,  2, 10,  2, 10,  2,  2,  2,  2,
        2,  2,  2,  2,  2,  2,  2,  2,  2,  2, 10,  2,  2,  2,  6,  2,  6,
        2,  6,  8,  6,  8,  8,  8,  8,  8,  8,  8,  8,  6,  6,  6,  8,  0,
        8,  8,  6,  8,  8,  8,  8,  6,  8,  8,  0,  8,  6,  8,  8,  8,  8,
        8,  6,  8,  0,  6,  6,  8,  0,  8,  0,  8,  8,  6,  0,  6,  0, 11,
        0,  0, 11,  6,  0, 11,  5,  0, 11, 13,  0,  0, 11,  5, 11, 11,  5,
       11,  5,  0, 13, 11,  0,  0,  5, 11,  5,  5, 11,  5, 11,  0,  5, 11,
        5, 13,  5,  0, 11, 11,  5,  5, 11,  5,  5,  5, 11,  5, 13,  5,  5,
        5, 11, 11,  5,  5,  0,  5,  5,  5,  5, 11, 12, 13,  5,  5, 11,  5,
       11, 12, 11, 11,  5, 12,  5, 11,  5,  5,  5,  5, 12,  5, 13,  5, 12,
        5, 12,  5,  5, 13,  5, 13,  5,  5, 12, 13,  1,  1,  5, 13, 12, 13,
        1, 12,  1, 12,  1

In [44]:
p_avg['KM']=clust.labels_

In [47]:
pd.set_option('display.max_rows', 20)
pd.set_option('display.min_rows', 20)
p_avg.loc[p_avg['KM']==4]

,Player,FG,FG%,3P,3P%,FT,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PTS,KM
92,Danny Green,1.000,0.414,1.000,0.512,0.125,1.000,0.250,0.625,0.875,0.250,0.375,0.125,0.750,3.125,4
405,Raul Neto,1.091,0.467,0.273,0.270,0.523,0.900,0.159,0.750,0.909,1.386,0.364,0.068,0.364,2.977,4
51,Buddy Boeheim,0.800,0.254,0.800,0.270,0.400,1.000,0.400,0.200,0.600,0.600,0.400,0.000,0.000,2.800,4
441,Svi Mykhailiuk,0.952,0.453,0.619,0.559,0.238,0.417,0.095,0.571,0.667,0.333,0.095,0.048,0.238,2.762,4
412,Robin Lopez,1.147,0.642,0.059,0.667,0.353,0.702,0.618,0.706,1.324,0.412,0.029,0.176,0.647,2.706,4
1,A.J. Lawson,1.000,0.467,0.538,0.426,0.077,0.500,0.154,0.615,0.769,0.077,0.077,0.000,0.077,2.615,4
452,Terry Taylor,1.143,0.484,0.143,0.139,0.179,0.750,0.786,0.607,1.393,0.357,0.107,0.179,0.321,2.607,4
182,Ish Smith,1.222,0.419,0.056,0.300,0.028,0.500,0.111,1.139,1.250,2.111,0.167,0.167,1.056,2.528,4
214,Jared Rhoden,1.000,0.496,0.167,0.111,0.333,1.000,1.000,1.000,2.000,0.000,0.167,0.000,0.000,2.500,4
212,James Johnson,1.000,0.315,0.214,0.250,0.286,0.625,0.143,1.357,1.500,0.714,0.286,0.429,0.357,2.500,4


In [48]:
p_avg.value_counts(['KM'])

KM
9     78
1     57
14    48
4     45
5     45
2     29
8     29
11    25
12    22
3     20
0     19
6     16
13    12
7      9
10     9
dtype: int64

In [53]:
len(KM_dict)

463

In [52]:
k = []
for p in p_idx:
    if p in KM_dict:
        k.append(KM_dict[p])
    else:
        k.append(16)
data['KM']= k

In [51]:
p_idx = data.Player

In [50]:
final = p_avg[['Player','KM']]
final.to_csv('KM_vals.csv')

In [49]:
KM_dict = dict(zip(p_avg.Player.values, p_avg.KM.values))
KM_dict

{'Joel Embiid': 7,
 'Luka Doncic': 7,
 'Damian Lillard': 7,
 'Giannis Antetokounmpo': 7,
 'Shai Gilgeous-Alexander': 7,
 'Jayson Tatum': 7,
 'Stephen Curry': 7,
 'LeBron James': 7,
 'Kevin Durant': 7,
 'Devin Booker': 3,
 'Donovan Mitchell': 3,
 'Kyrie Irving': 3,
 'Ja Morant': 3,
 'Trae Young': 3,
 'Jaylen Brown': 3,
 'Anthony Davis': 10,
 'Zion Williamson': 3,
 "De'Aaron Fox": 3,
 'Lauri Markkanen': 10,
 'Julius Randle': 10,
 'DeMar DeRozan': 3,
 'Anthony Edwards': 3,
 'Zach LaVine': 3,
 'Nikola Jokic': 10,
 'Pascal Siakam': 3,
 'Kawhi Leonard': 3,
 'Jalen Brunson': 3,
 'Paul George': 3,
 'Bradley Beal': 3,
 'LaMelo Ball': 3,
 'Brandon Ingram': 3,
 'Kristaps Porzingis': 10,
 'Jimmy Butler': 3,
 'Klay Thompson': 2,
 'James Harden': 3,
 'Darius Garland': 2,
 'Keldon Johnson': 2,
 'Jalen Green': 2,
 'Bojan Bogdanovic': 2,
 'Terry Rozier': 2,
 'Kyle Kuzma': 10,
 'Anfernee Simons': 2,
 'CJ McCollum': 2,
 'Desmond Bane': 2,
 'Bam Adebayo': 10,
 'Jordan Clarkson': 2,
 'Karl-Anthony Towns': 